In [1]:
import sys,os, json, shutil, filecmp
from pathlib import Path
import numpy as np
import pandas as pd

## Parameters

In [10]:
overwrite = 0
nTTs = 16

## Path to Directory

In [11]:
# path to cluster (unix)
animal = 'Li'
date = '062818'
task = 'T3g'
cluster_experimenter = 'AG'
clusterDir = Path('/mnt/c/Users/alexg8/Documents/Data/'+ 
                   animal+'/Clustered/')
assert clusterDir.exists(), 'Cluster Directory not Found'

sessionName = animal+'_'+task+'_'+date+'_KSClusters'
assert (clusterDir / sessionName).exists(), 'Session Not Found'


## Get Animals Json cluster summary

In [13]:
Cl_Summary_fn = animal+'_ClusteringSummary.json'
if (clusterDir/Cl_Summary_fn).exists():
    with open(str(clusterDir/Cl_Summary_fn), 'r') as f:
        cluster_summary = json.load(f)
    if not date in cluster_summary[animal].keys() or overwrite:
        cluster_summary[animal][date]={}
    if not task in cluster_summary[animal][date].keys() or overwrite:
        cluster_summary[animal][date][task] = {}
    else:
        print('Warning clustering data exists for {}, and overwrite = false'.format(sessionName))
else:
    cluster_summary = {}
    cluster_summary[animal] = {}
    cluster_summary[animal][date]={}
    cluster_summary[animal][date][task] = {}


Warning clustering data exists for Li_T3g_062818_KSClusters, and overwrite = false


{'T3g': {'nCells': 10,
  'nMua': 16,
  'cell_IDs': {'1': [],
   '2': [],
   '3': [1, 6],
   '4': [],
   '5': [],
   '6': [],
   '7': [3],
   '8': [4, 10],
   '9': [2],
   '10': [0, 6, 8],
   '11': [],
   '12': [],
   '13': [],
   '14': [9],
   '15': [],
   '16': []},
  'mua_IDs': {'1': [7],
   '2': [2, 10],
   '3': [0, 3],
   '4': [2],
   '5': [],
   '6': [3],
   '7': [],
   '8': [3],
   '9': [11],
   '10': [4, 5, 7, 9],
   '11': [],
   '12': [],
   '13': [],
   '14': [1, 8, 10],
   '15': [],
   '16': []}}}

## Get Files and Update Json File

In [203]:
cluster_summary[animal][date][task]['nCells'] = 0
cluster_summary[animal][date][task]['nMua'] = 0
cluster_summary[animal][date][task]['cell_IDs'] = {}
cluster_summary[animal][date][task]['mua_IDs'] = {}
for tt in np.arange(1,nTTs+1):
    cluster_summary[animal][date][task]['cell_IDs'][int(tt)]=[]
    cluster_summary[animal][date][task]['mua_IDs'][int(tt)]=[]
    fn = clusterDir/sessionName/('tt_'+str(tt))/'cluster_group.tsv'
    assert fn.exists(), 'could not find record for tt {}; in {}'.format(tt,sessionName)
    d=pd.read_csv(fn,delimiter='\t')
    cells = np.where(d['group']=='good')[0].tolist()
    mua = np.where(d['group']=='mua')[0].tolist()
    for cc in cells:
        cluster_summary[animal][date][task]['cell_IDs'][int(tt)].append(cc)
    for mm  in mua:
        cluster_summary[animal][date][task]['mua_IDs'][int(tt)].append(mm)
    cluster_summary[animal][date][task]['nCells'] += len(cells)
    cluster_summary[animal][date][task]['nMua'] += len(mua) 

with open(str(clusterDir/Cl_Summary_fn), 'w') as f:
        json.dump(cluster_summary, f, indent=4)

In [204]:
print("Results for {}:\n nCells = {} \n nMuas = {}"
      .format(sessionName,cluster_summary[animal][date][task]['nCells'],
              cluster_summary[animal][date][task]['nMua']))


Results for Li_T3g_062818_KSClusters:
 nCells = 10 
 nMuas = 16


In [29]:
n=0
m=0
for d in cluster_summary[animal].keys():
    n+=cluster_summary[animal][d][task]['nCells']
    m+=cluster_summary[animal][d][task]['nMua']
print(" nSessions = {} \n nCells = {} \n nMua = {}".format(len(cluster_summary[animal].keys()),n,m))


 nSessions = 10 
 nCells = 86 
 nMua = 145


In [237]:
sessionName.split('_')

['Li', 'T3g', '062818', 'KSClusters']

In [206]:
cluster_summary[animal].keys()

dict_keys(['060118', '060418', '061518', '061318', '061818', '062018', '062518', '062618', '062718', '062818'])

## copy results to oak (note: oak must be mounted on the ubuntu shell: )
### sudo mkdir /mnt/o
### sudo mount -t drvfs O: /mnt/o
### to unmount
### sudo unmount /mnt/o

In [231]:
oakPath = Path('/mnt/o/giocomo/alexg/Clustered/'+animal)
clusterDir = Path('/mnt/c/Users/alexg8/Documents/Data/'+ 
                   animal+'/Clustered/')
# copy summary
if not filecmp.cmp(str(clusterDir/Cl_Summary_fn),str(oakPath/Cl_Summary_fn),shallow=True):
    shutil.copyfile(str(clusterDir/Cl_Summary_fn),str(oakPath/Cl_Summary_fn))
    print('updated file summary file')
else:
    print('summary file is the same, skipping copy.')
# copy individual tetrode clusters
notUpDatedList = []
for date in cluster_summary[animal].keys():
    sessionName = animal+'_'+task+'_'+date+'_KSClusters'
    notUpDatedList = []
    for tt in np.arange(1,nTTs+1):
        fn = clusterDir/sessionName/('tt_'+str(tt))/'cluster_group.tsv'
        sp = oakPath/sessionName/('tt_'+str(tt))/'cluster_group.tsv'
        if not filecmp.cmp(str(fn),str(sp),shallow=True):
            shutil.copyfile(str(fn),str(sp))
        else:
            notUpDatedList.append(tt)
            #print('{} {} Cluster file exists and it is the same, not updating'.format(sessionName,tt))
    if len(notUpDatedList)==16:
        print("{}: All tetrodes have already been clustered. ".format(sessionName))
    elif len(notUpDatedList)==0:
        print("{}: Updated all tetrode clusters".format(sessionName))
    else:
        print("{}: Indetical cluster files, no updates for TTs {}".format(sessionName, notUpDatedList))
    

summary file is the same, skipping copy.
Li_T3g_060118_KSClusters: All tetrodes had already been clustered. 
Li_T3g_060418_KSClusters: All tetrodes had already been clustered. 
Li_T3g_061518_KSClusters: All tetrodes had already been clustered. 
Li_T3g_061318_KSClusters: All tetrodes had already been clustered. 
Li_T3g_061818_KSClusters: All tetrodes had already been clustered. 
Li_T3g_062018_KSClusters: All tetrodes had already been clustered. 
Li_T3g_062518_KSClusters: All tetrodes had already been clustered. 
Li_T3g_062618_KSClusters: All tetrodes had already been clustered. 
Li_T3g_062718_KSClusters: All tetrodes had already been clustered. 
Li_T3g_062818_KSClusters: All tetrodes had already been clustered. 


In [21]:
oakPath = Path('/mnt/o/giocomo/alexg/Clustered/'+animal)

[PosixPath('/mnt/o/giocomo/alexg/Clustered/Li/Li_ClusteringSummary.json')]

In [32]:
for i in oakPath.glob('*'):
    print(i.name.split('_'))

['Li', 'ClusteringSummary.json']
['Li', 'OF', '060118', 'KSClusters']
['Li', 'OF', '060418', 'KSClusters']
['Li', 'OF', '060518', 'KSClusters']
['Li', 'OF', '060718', 'KSClusters']
['Li', 'OF', '060818', 'KSClusters']
['Li', 'OF', '061118', 'KSClusters']
['Li', 'OF', '061218', 'KSClusters']
['Li', 'OF', '061318', 'KSClusters']
['Li', 'OF', '061418', 'KSClusters']
['Li', 'OF', '061518', 'KSClusters']
['Li', 'OF', '061818', 'KSClusters']
['Li', 'OF', '062018', 'KSClusters']
['Li', 'OF', '062518', 'KSClusters']
['Li', 'OF', '062618', 'KSClusters']
['Li', 'OF', '062718', 'KSClusters']
['Li', 'OF', '062818', 'KSClusters']
['Li', 'OF', '062918', 'KSClusters']
['Li', 'OF', '070218', 'KSClusters']
['Li', 'OF', '070618', 'KSClusters']
['Li', 'OF', '071018', 'KSClusters']
['Li', 'OF', '080118', 'KSClusters']
['Li', 'OF', '080218', 'KSClusters']
['Li', 'T3g', '060118', 'KSClusters']
['Li', 'T3g', '060418', 'KSClusters']
['Li', 'T3g', '061318', 'KSClusters']
['Li', 'T3g', '061518', 'KSClusters']
[